# Car accidents in Latvia

The main goal of this notebook is to extract and manipulate Latvian car accident data for further data visualization. 

- Link to the data: https://gis.ic.iem.gov.lv/giswebcais/
- Region: All Latvia
- Date range: 2015-2021

In [ ]:
#############################################################################
#####
#####         1. Import necessary packages
#####
#############################################################################

import pandas as pd
import csv
import datetime
import numpy as np
import seaborn as sns
from tqdm import tqdm
tqdm.pandas()
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [ ]:
#############################################################################
#####
#####         2. Load and manipulate Accidents dataset
#####
#############################################################################

# Read data from csv
accidents = pd.read_csv('202101100758_CaisVirsnot.csv', sep = ';')

# Sum the number of total people involved in the accident
list_name =['To skaitā velosipēdi','Iesaistīto gājēju skaits','Bojāgājušo skaits','Personas ar smagiem miesas bojājumiem','Personas ar viegliem miesas bojājumiem','Personas bez miesas bojājumiem','Personu skaits reibumā','Bojāgājušo skaits, kas nav sasnieguši 18 gadus','Personas ar miesas bojājumiem, kas nav sasniegušas 18 gadus']
accidents['people_involved']=accidents.loc[:,list_name].sum(axis=1)

# # Add column with the year of the accident, based on the accident date
accidents['year'] = pd.to_datetime(accidents['Notikuma datums un laiks'], format="%d.%m.%Y. %H:%M:%S").dt.year

# Add column with the month of the accident, based on the accident date
accidents['month'] = pd.to_datetime(accidents['Notikuma datums un laiks'], format="%d.%m.%Y. %H:%M:%S").dt.month

# Add column with the year month of the accident, based on the accident date
accidents['year_month'] = pd.to_datetime(accidents['Notikuma datums un laiks'],format="%d.%m.%Y. %H:%M:%S").dt.to_period('M')

# Add column with the hour without minutes of the accident, based on the accident date
accidents['hour'] = pd.to_datetime(accidents['Notikuma datums un laiks'], format="%d.%m.%Y. %H:%M:%S").dt.hour

# Define function to classify each accident by part of the day when it happened
def part_of_day(x):
    if (x > 6) and (x <= 8):
        return 'Early Morning'
    elif (x > 8) and (x <= 12 ):
        return 'Morning'
    elif (x > 12) and (x <= 16):
        return'Noon'
    elif (x > 16) and (x <= 20) :
        return 'Eve'
    elif (x > 20) and (x <= 24):
        return'Night'
    elif (x <= 6):
        return'Late Night'

# Assign part of the day to each accident, based on the function we defined    
accidents['part_of_day'] = accidents['hour'].apply(part_of_day)

# Define function to classify each accident by category of the injury
def accident_category(accidents):
    if accidents['Bojāgājušo skaits'] > 0 or accidents['Bojāgājušo skaits, kas nav sasnieguši 18 gadus'] > 0 :
        return 'Fatal'
    elif accidents['Personas ar smagiem miesas bojājumiem'] > 0 or accidents['Personas ar miesas bojājumiem, kas nav sasniegušas 18 gadus'] > 0 :
        return 'Severe'
    elif accidents['Personas ar viegliem miesas bojājumiem'] > 0 :
        return 'Minor'
    else:
        return 'Without Injury'

# Assign category of the injury to each accident
accidents['accident_category'] = accidents.apply(accident_category, axis=1)

# If there are null values on the Novads field, fill it with Pilseta's field (main cities are not associated to any Novads)
accidents['Novads'] = accidents['Novads'].fillna(accidents.Pilsēta)

In [ ]:
#############################################################################
#####
#####         3. Load and manipulate accident participants dataset
#####
#############################################################################

# Read data from csv
accidents_participants = pd.read_csv('202101100758_Dalibnieki.csv', sep = ';')

# Change data type
accidents_participants['Izlaiduma gads'] = accidents_participants['Izlaiduma gads'].astype(str).str[:4].astype('float')

# Let's get the accident year from accidents dataset and join it to accidents_participants
# We'll use it to calculate how old is the car at the year of the event
year_accident = accidents[['Notikuma identifikators','year']]
accidents_participants = accidents_participants.merge(year_accident, how = 'left', on ='Notikuma identifikators')
accidents_participants['year'] = accidents_participants.year.astype('float')
accidents_participants['car_years_old'] = accidents_participants['year'] - accidents_participants['Izlaiduma gads']

# Asign tier categories to each car involved in an accident. This will help us visualize the data
conditions = [
    (accidents_participants['car_years_old'] <= 10),
    (accidents_participants['car_years_old'] > 10) & (accidents_participants['car_years_old'] <= 20),
    (accidents_participants['car_years_old'] > 20) & (accidents_participants['car_years_old'] <= 30),
    (accidents_participants['car_years_old'] > 30)
    ]

# create a list of the values we want to assign for each condition
values = ['cars_involved_less_than_10yrs_old', 'cars_involved_20-10yrs_old', 'cars_involved_30-20yrs_old', 'cars_involved_more_than_30yrs_old']

# create a new column and use np.select to assign values to it using our lists as arguments
accidents_participants['car_years_old_tier'] = np.select(conditions, values)

# Join year_month to ease the visualization linkage
accidents_participants = accidents_participants.merge(accidents[['year_month', 'Notikuma identifikators']], on='Notikuma identifikators', how='left')

In [ ]:
#############################################################################
#####
#####         4. Create dataset based on intoxicated drivers
#####
#############################################################################

# Filter out data from accidents_participants dataset
intoxicated = accidents_participants.loc[(accidents_participants['Reibums']=='Jā') & (accidents_participants['Statuss']=='Vadītājs')]

# Count number of intoxicated drivers per accident
intoxicated = intoxicated[['Notikuma identifikators', 'Reibums']].groupby('Notikuma identifikators').count()

# Assign boolean category: intoxicated drivers TRUE/FALSE
intoxicated['IntoxicatedInvolved'] = intoxicated['Reibums'] > 0

# Join intoxicated dataset to the accidents dataset
accidents = accidents.merge(intoxicated, how = 'left', on ='Notikuma identifikators')

# Fill missing values with 0 to avoid errors
accidents.Reibums.fillna(0, inplace=True)

In [ ]:
#############################################################################
#####
#####         5. Create dataset with calculated metrics per year and month
#####
#############################################################################

accidents_metrics = pd.DataFrame()
accidents_metrics['accidents_per_year_month'] = accidents.groupby('year_month')['Notikuma identifikators'].nunique()
accidents_metrics['accidents_intoxicated_per_year_month'] = accidents[accidents['Reibums'] > 0].groupby('year_month')['Notikuma identifikators'].nunique()
accidents_metrics['accidents_bicycles_per_year_month'] = accidents[accidents['To skaitā velosipēdi'] > 0].groupby('year_month')['Notikuma identifikators'].nunique()
accidents_metrics['accidents_pedestrians_per_year_month'] = accidents[accidents['Iesaistīto gājēju skaits'] > 0].groupby('year_month')['Notikuma identifikators'].nunique()
accidents_metrics['accident_intoxicated_rate'] = accidents_metrics['accidents_intoxicated_per_year_month'] / accidents_metrics['accidents_per_year_month']

In [ ]:
#############################################################################
#####
#####         6. Prepare data for visualization: only between 2015 and 2020
#####
#############################################################################

# Exclude values from 2021
accidents = accidents[accidents['year']<2021]
accidents_participants = accidents_participants[accidents_participants['year']<2021]

In [ ]:
#############################################################################
#####
#####         7. Save data to Excel format for visualization
#####
#############################################################################

# Example of path - change to yours!
toExcel = pd.ExcelWriter("/Users/UserName/Desktop/" + 'latvia_traffic_accidents.xlsx', engine='xlsxwriter')
accidents.to_excel(toExcel, sheet_name='accidents')
accidents_participants.to_excel(toExcel, sheet_name='accidents_participants')
accidents_metrics.to_excel(toExcel, sheet_name='accidents_metrics')
toExcel.save()